In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Load a sample time series dataset (Apple stock prices)
url = "https://query1.finance.yahoo.com/v7/finance/download/AAPL?period1=1609459200&period2=1640995200&interval=1d&events=history"
df = pd.read_csv(url)

# Display dataset
print("Dataset Head:")
print(df.head())

# Plot closing prices
plt.figure(figsize=(10, 5))
plt.plot(df["Date"], df["Close"], label="Closing Price")
plt.xlabel("Date")
plt.ylabel("Price")
plt.title("Apple Stock Prices")
plt.legend()
plt.show()

In [ ]:
# Use only the 'Close' price for prediction
data = df['Close'].values.reshape(-1, 1)

# Scale the data between 0 and 1
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# Function to create sequences of data for LSTM input
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length])
    return np.array(X), np.array(y)

# Define sequence length
SEQ_LENGTH = 50
X, y = create_sequences(scaled_data, SEQ_LENGTH)

# Split into train and test sets (80-20 split)
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

print(f"Training Data Shape: {X_train.shape}")
print(f"Testing Data Shape: {X_test.shape}")

In [ ]:
# Build the LSTM model
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(SEQ_LENGTH, 1)),
    LSTM(50),
    Dense(1)  # Predict the next value
])

# Compile the model
model.compile(optimizer="adam", loss="mean_squared_error")

# Display model summary
model.summary()

In [ ]:
# Train the LSTM model
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

In [ ]:
# Plot training and validation loss
plt.figure(figsize=(8, 5))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

In [ ]:
# Predict on the test set
predicted_prices = model.predict(X_test)

# Inverse transform predictions and true values
predicted_prices = scaler.inverse_transform(predicted_prices)
true_prices = scaler.inverse_transform(y_test.reshape(-1, 1))

# Plot the true and predicted values
plt.figure(figsize=(10, 5))
plt.plot(true_prices, label="True Prices", color="blue")
plt.plot(predicted_prices, label="Predicted Prices", color="red")
plt.title("LSTM Time Series Forecasting")
plt.xlabel("Time")
plt.ylabel("Price")
plt.legend()
plt.show()

In [ ]:
# Forecast future values
future_days = 10
future_predictions = []

# Use the last available sequence
current_input = X_test[-1]

for _ in range(future_days):
    next_prediction = model.predict(current_input.reshape(1, SEQ_LENGTH, 1))[0]
    future_predictions.append(next_prediction)
    current_input = np.append(current_input[1:], next_prediction).reshape(SEQ_LENGTH, 1)

# Inverse transform the predictions
future_predictions = scaler.inverse_transform(future_predictions)

# Display predictions
print(f"Next {future_days} days predicted prices:")
print(future_predictions)